# Web Scraping

In [ ]:
import time

import bs4
import pyodbc
import requests
import pandas as pd

In [1]:
def consult(url):
    # Gets content from page
    try:
        req = requests.get(url) 
        return req.content      
    except:
        pass


def capture_html(url):
    # Returns the HTML page parsed
    html = consult(url)
    soup = bs4.BeautifulSoup(html, "html.parser")
    return soup


def get_html_header(html):
    # Gets the headers from the HTML page
    dt = html.find_all("dt")
    dd = html.find_all("dd")
    values = {}
    for key, val in zip(dt, dd):
        values[key.get_text()] = val.get_text()

    return values


def get_proposition(proposition_id, year, prop_type, prop_name):
    # Gets the remaining values for the proposition
    url = f"https://www.legislador.com.br//LegisladorWEB.ASP?WCI=ProposicaoTexto&ID=9&TPProposicao={prop_type}&nrProposicao={proposition_id}&aaProposicao={year}"
    html = capture_html(url)
    values = get_html_header(html)
    values["Proposição"] = proposition_id
    values["Ano"] = year
    values["Tipo"] = prop_name
    values["Texto"] = html.p.get_text()

    return values

In [2]:
def sql_select(query):
    # Using SSMS with Windows' authentication
    conn = pyodbc.connect("Trusted_Connection=yes",
                            driver="{ODBC Driver 17 for SQL Server}",
                            server="localhost",
                            database="PG")
    res = pd.read_sql_query(query, conn)
    return res


def sql_truncate(table_name):
    # Truncates the table with the given name
    conn = pyodbc.connect("Trusted_Connection=yes",
                        driver="{ODBC Driver 17 for SQL Server}",
                        server="localhost",
                        database="PG")

    cursor = conn.cursor()
    cursor.execute(f"""
            TRUNCATE TABLE {table_name}
        """)

    conn.commit()
    cursor.close()


def rename_columns_to_db(dataframe):
    # Rename dataframe's columns to match the database's columns
    renamed = dataframe.rename(columns={
                                        "Proposição": "proposition",
                                        "Ano": "year", "Texto": "text",
                                        "Reunião": "date_meeting",
                                        "Deliberação": "date_deliberation",
                                        "Situação": "situation", "Assunto": "subject",
                                        "Autor": "author", "Tipo": "type"
                                    })
    
    return renamed


def insert_propositions(dataframe, database = "PG", table_name = "Propositions"):
    # Inserts the read proposition into the database
    dataframe = rename_columns_to_db(dataframe)
    conn_df = pd.DataFrame(columns=["date_meeting", "date_deliberation",
                                    "situation", "subject", "author", "proposition",
                                    "year", "text", "type"])
    conn_df = conn_df.append(dataframe).fillna("")
    conn_df["date_meeting"] = pd.to_datetime(conn_df["date_meeting"], format="%d/%m/%Y")
    conn_df["date_deliberation"] = pd.to_datetime(conn_df["date_deliberation"], format="%d/%m/%Y")

    conn = pyodbc.connect("Trusted_Connection=yes",
                        driver="{ODBC Driver 17 for SQL Server}",
                        server="localhost",
                        database=database)

    cursor = conn.cursor()

    for _, row in conn_df.iterrows():
        cursor.execute(f"""INSERT INTO {table_name} (
            date_meeting, date_deliberation, situation, subject, author, proposition, year, text, type
        ) VALUES(?, ?, ?, ?, ?, ?, ?, ?, ?)""",
            row["date_meeting"], row["date_deliberation"], row["situation"], row["subject"],
            row["author"], row["proposition"], row["year"], row["text"], row["type"]
        )

    conn.commit()


def get_next_proposition(year, prop_type, prop_name):
    # Gets the last proposition
    year_data = sql_select(f"""SELECT proposition = max(proposition)
                           FROM Propositions
                           WHERE year = {year} AND type = '{prop_name}'""")
    last_proposition = year_data["proposition"].loc[0]

    # Verifies if a proposition already exists in current year and type
    if last_proposition == None:
        next_proposition = 1
    else:
        next_proposition = int(last_proposition) + 1 

    # Gets the next proposition using scraping
    data = get_proposition(next_proposition, year, prop_type, prop_name)
    table = pd.DataFrame([data])
    insert_propositions(table)


def get_propositions(year, prop_type, prop_name, max_error = 10, wait_secs = 0.25):
    # Gets all propositions from a given year and type
    errors_count = 0

    # When no proposition is found, error will start increasing
    while errors_count < max_error:
        try:
            get_next_proposition(year, prop_type, prop_name)
        except:
            errors_count += 1
            pass

        time.sleep(wait_secs)


def insert_db(start_year, end_year):
    # Inserts propositions between a range of years
    types = {
        1: "Indicação",
        2: "Requerimento",
        3: "Moção",
        4: "Resolução MD",
        5: "Portaria"
    }

    for i in range(start_year, end_year+1):
        print(f"Inserting data from {i}")
        for key in types:
            try:
                get_propositions(i, key, types[key])
            except:
                pass
        print(f"Finished year {i}")

In [ ]:
insert_db(1995, 2023)